In [ ]:
# by caleb fried 04/2022

#initilization
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import savemat
import os
import h5py as h5
import time
ax = {'x':0,'y':1,'z':2}
#define a dict of axes to make things simpler

In [ ]:
from auspex.instruments import SR830
from pyanc350.v4 import Positioner

In [ ]:
lockin = SR830()
lockin.connect('GPIB0::20')   
anc = Positioner()
#instantiate positioner as anc
print('-------------------------------------------------------------')
print('capacitances:')
for axis in sorted(ax.keys()):
    print(axis,anc.measureCapacitance(ax[axis]))
    anc.setFrequency(ax[axis],200)
    anc.setAmplitude(ax[axis],45)

In [ ]:
# assume we start around 1 mm below the large silicon part
step1 = 0
step2 = 0
data = [0] * 100000
print('starting scan at',anc.getPosition(ax['y']))
anc.startContinuousMove(ax['y'],1,1) # assuming right is forward
while step1 == 0:
    i = i+1
    time.sleep(0.01)
    data[i] = lockin.x
    if (data[i] - data[i-2])/data[i-2] < - 0.1:
        step1 = 1
anc.startContinuousMove(ax['y'],0,1)
print('reached Si at',anc.getPosition(ax['y']))
input("Press Enter to continue...")

anc.startContinuousMove(ax['z'],1,0)
while step2 == 0:
    i = i+1
    time.sleep(0.01)
    data[i] = lockin.x
    if (data[i] - data[i-2])/data[i-2] > 0.1:
        step2 = 1
anc.startContinuousMove(ax['z'],0,0)
print('reached corner at',anc.getPosition(ax['z']))
input("Press Enter to continue...")

print('-------------------------------------------------------------')
print('moving to CPW')
start = anc.getPosition(['y'])                         
anc.setAxisOutput(ax['y'], 1, 0)
anc.setTargetRange(ax['y'],1e-6)
anc.setTargetPosition(ax['y'], start - 2.00863e-3) # assuming right is forward
anc.startAutoMove(ax['y'], 1, 0)

#check what's happening
time.sleep(0.5)
moving = 1
target = 0
while target == 0:
    connected, enabled, moving, target, eotFwd, eotBwd, error = anc.getAxisStatus(ax['y']) #find bitmask of status
    if target == 0:
        print('axis moving, currently at',anc.getPosition(ax['y']))
    elif target == 1:
        print('axis arrived at',anc.getPosition(ax['y']))
        anc.startAutoMove(ax['y'], 0, 0)
    time.sleep(0.5)
input("Press Enter to continue...")

print('-------------------------------------------------------------')
print('moving to sample area')
start = anc.getPosition(['z'])                         
anc.setAxisOutput(ax['z'], 1, 0)
anc.setTargetRange(ax['z'],1e-6)
anc.setTargetPosition(ax['z'], start - 2.685e-3) #assuming positive y is right
anc.startAutoMove(ax['z'], 1, 0)

#check what's happening
time.sleep(0.5)
moving = 1
target = 0
while target == 0:
    connected, enabled, moving, target, eotFwd, eotBwd, error = anc.getAxisStatus(ax['z']) #find bitmask of status
    if target == 0:
        print('axis moving, currently at',anc.getPosition(ax['z']))
    elif target == 1:
        print('axis arrived at',anc.getPosition(ax['z']))
        anc.startAutoMove(ax['z'], 0, 0)
    time.sleep(0.5)
print('end position: y =',anc.getPosition(ax['y']),', z ='anc.getPosition(ax['z']))
